# 🔍 BACEN Data Validation & Calculation Audit
## Comprehensive Step-by-Step Analysis for Banco Insights 2.0

**Purpose**: Identify discrepancies between CSV source data and database/API calculations

**Issue**: System reports R$ 148.3B total assets vs expected R$ 16.9T (16,906.58B)

**Date**: August 2025

---

## 📋 Step 1: Environment Setup & Data Import

In [1]:
import pandas as pd
import numpy as np
import psycopg2
from datetime import datetime
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# Load environment variables for database connection
load_dotenv('/Users/iagoaffonso/code/IagoAffonso/banco-insights-2.0/supabase_config.env')

print("✅ Libraries imported successfully")
print(f"📅 Analysis started at: {datetime.now()}")

✅ Libraries imported successfully
📅 Analysis started at: 2025-08-10 21:22:21.372495


## 📊 Step 2: Load Raw CSV Data

In [2]:
# Path to the main CSV file containing financial metrics
csv_path = '/Users/iagoaffonso/code/IagoAffonso/banco-insights-2.0/bacen_project_v1/data/financial_metrics.csv'

print(f"📁 Loading data from: {csv_path}")
print(f"📏 File size: {os.path.getsize(csv_path) / (1024*1024):.2f} MB")

# Load with sample first to understand structure
sample_df = pd.read_csv(csv_path, nrows=1000)
print(f"\n📋 CSV Structure (first 1000 rows):")
print(f"Columns: {list(sample_df.columns)}")
print(f"Shape: {sample_df.shape}")
print(f"\nData types:")
print(sample_df.dtypes)

📁 Loading data from: /Users/iagoaffonso/code/IagoAffonso/banco-insights-2.0/bacen_project_v1/data/financial_metrics.csv
📏 File size: 1110.75 MB

📋 CSV Structure (first 1000 rows):
Columns: ['TipoInstituicao', 'CodInst', 'AnoMes', 'NomeRelatorio', 'NumeroRelatorio', 'Grupo', 'Conta', 'NomeColuna', 'DescricaoColuna', 'Saldo', 'AnoMes_M', 'AnoMes_Q', 'AnoMes_Y', 'NomeRelatorio_Grupo_Coluna', 'NomeInstituicao']
Shape: (1000, 15)

Data types:
TipoInstituicao                 int64
CodInst                        object
AnoMes                         object
NomeRelatorio                  object
NumeroRelatorio                 int64
Grupo                          object
Conta                           int64
NomeColuna                     object
DescricaoColuna                object
Saldo                         float64
AnoMes_M                       object
AnoMes_Q                       object
AnoMes_Y                        int64
NomeRelatorio_Grupo_Coluna     object
NomeInstituicao           

In [3]:
# Load the complete dataset
print("🔄 Loading complete financial_metrics dataset...")
financial_metrics = pd.read_csv(csv_path)

print(f"✅ Complete dataset loaded:")
print(f"Shape: {financial_metrics.shape}")
print(f"Memory usage: {financial_metrics.memory_usage(deep=True).sum() / (1024*1024):.2f} MB")

# Display first few rows
print("\n🔍 First 5 rows:")
financial_metrics.head()

🔄 Loading complete financial_metrics dataset...
✅ Complete dataset loaded:
Shape: (2682551, 15)
Memory usage: 2856.80 MB

🔍 First 5 rows:


,TipoInstituicao,CodInst,AnoMes,NomeRelatorio,NumeroRelatorio,Grupo,Conta,NomeColuna,DescricaoColuna,Saldo,AnoMes_M,AnoMes_Q,AnoMes_Y,NomeRelatorio_Grupo_Coluna,NomeInstituicao
0,2,C0051389,2014-12-01,Demonstração de Resultado,4,Resultado de Intermediação Financeira - Receit...,78204,Rendas de Operações de Arrendamento Mercantil ...,[71200004]+[81940000],819733.58,2014-12,2014Q4,2014,Demonstração de Resultado_Resultado de Interme...,BONCRED
1,2,C0051389,2014-12-01,Demonstração de Resultado,4,Resultado de Intermediação Financeira - Receit...,78205,Rendas de Operações com TVM \n(a3),[71400000]+[71500003]-[71580009]+[71940003]+[7...,828141.53,2014-12,2014Q4,2014,Demonstração de Resultado_Resultado de Interme...,BONCRED
2,2,C0051389,2014-12-01,Demonstração de Resultado,4,Resultado de Intermediação Financeira - Receit...,78206,Rendas de Operações com Instrumentos Financeir...,[71580009]+[81550005]+[71990266]+[81830268],0.00,2014-12,2014Q4,2014,Demonstração de Resultado_Resultado de Interme...,BONCRED
3,2,C0051389,2014-12-01,Demonstração de Resultado,4,Resultado de Intermediação Financeira - Receit...,78207,Resultado de Operações de Câmbio \n(a5),[71300007]+[81400007],0.00,2014-12,2014Q4,2014,Demonstração de Resultado_Resultado de Interme...,BONCRED
4,2,C0051389,2014-12-01,Demonstração de Resultado,4,Resultado de Intermediação Financeira - Receit...,78231,Rendas de Aplicações Compulsórias \n(a6),[71955005]+[71960007]+[71965002]+[71990125]+[8...,0.00,2014-12,2014Q4,2014,Demonstração de Resultado_Resultado de Interme...,BONCRED


## 🔎 Step 3: Explore Data Structure & Key Columns

In [4]:
# Analyze key columns for filtering
print("🔍 Analyzing key columns for filtering:")
print("\n1. NomeRelatorio unique values:")
print(financial_metrics['NomeRelatorio'].value_counts())

print("\n2. AnoMes_Q unique values (last 10):")
print(financial_metrics['AnoMes_Q'].value_counts().tail(10))

print("\n3. NomeColuna containing 'Ativo Total':")
ativo_columns = financial_metrics[financial_metrics['NomeColuna'].str.contains('Ativo Total', na=False)]['NomeColuna'].unique()
print(ativo_columns)

print("\n4. NomeColuna containing 'Carteira de Crédito':")
credit_columns = financial_metrics[financial_metrics['NomeColuna'].str.contains('Carteira de Crédito', na=False)]['NomeColuna'].unique()
print(credit_columns)

🔍 Analyzing key columns for filtering:

1. NomeRelatorio unique values:
NomeRelatorio
Demonstração de Resultado                                            2062879
Resumo                                                                569848
Carteira de crédito ativa - quantidade de clientes e de operações      49824
Name: count, dtype: int64

2. AnoMes_Q unique values (last 10):
AnoMes_Q
2019Q3    52460
2021Q4    52436
2020Q3    51884
2019Q4    51864
2021Q2    51851
2020Q4    51810
2021Q3    51806
2021Q1    51730
2020Q1    51660
2020Q2    51657
Name: count, dtype: int64

3. NomeColuna containing 'Ativo Total':
['Ativo Total']

4. NomeColuna containing 'Carteira de Crédito':
['Carteira de Crédito Classificada']


## 📈 Step 4: Replicate User's Calculation (Ativo Total)

In [6]:
# Replicate the exact calculation provided by the user
print("🧮 Replicating user's Ativo Total calculation:")
print("Filter: NomeRelatorio == 'Resumo' AND AnoMes_Q == '2024Q3' AND NomeColuna == 'Ativo Total'")

# Step by step filtering
step1 = financial_metrics[financial_metrics['NomeRelatorio'] == 'Resumo']
print(f"📊 After NomeRelatorio filter: {step1.shape[0]:,} rows")

step2 = step1[step1['AnoMes_Q'] == '2024Q3']
print(f"📊 After AnoMes_Q filter: {step2.shape[0]:,} rows")

step3 = step2[step2['NomeColuna'] == 'Ativo Total']
print(f"📊 After NomeColuna filter: {step3.shape[0]:,} rows")

# Calculate the sum
sumativo_total_2024Q3 = step3['Saldo'].sum()
print(f"\n💰 ATIVO TOTAL 2024Q3: {sumativo_total_2024Q3:,.2f}")
print(f"💰 In billions: {sumativo_total_2024Q3/1_000_000_000_000:.2f} trillion")
print(f"💰 In billions: {sumativo_total_2024Q3/1_000_000_000:.2f} billion")

# Show sample of the filtered data
print("\n🔍 Sample of filtered data:")
print(step3[['CodInst', 'Saldo']].head(10))

🧮 Replicating user's Ativo Total calculation:
Filter: NomeRelatorio == 'Resumo' AND AnoMes_Q == '2024Q3' AND NomeColuna == 'Ativo Total'
📊 After NomeRelatorio filter: 569,848 rows
📊 After AnoMes_Q filter: 11,160 rows
📊 After NomeColuna filter: 1,395 rows

💰 ATIVO TOTAL 2024Q3: 16,906,576,545,732.06
💰 In billions: 16.91 trillion
💰 In billions: 16906.58 billion

🔍 Sample of filtered data:
          CodInst         Saldo
2256996  C0031859  2.820279e+10
2257002  C0031873  1.679698e+10
2257008  C0031976  5.561670e+10
2257014  C0032119  2.324477e+10
2257020  C0041856  6.744691e+10
2257026  C0049906  2.442453e+12
2257032  C0049944  6.158789e+11
2257038  C0050304  2.554887e+10
2257044  C0050524  4.311332e+08
2257050  C0050940  7.218484e+09


## 💳 Step 5: Replicate User's Calculation (Carteira de Crédito)

In [7]:
# Replicate credit portfolio calculation
print("🧮 Replicating user's Carteira de Crédito calculation:")
print("Filter: NomeRelatorio == 'Resumo' AND AnoMes_Q == '2024Q3' AND NomeColuna == 'Carteira de Crédito Classificada'")

# Step by step filtering
credit_step1 = financial_metrics[financial_metrics['NomeRelatorio'] == 'Resumo']
credit_step2 = credit_step1[credit_step1['AnoMes_Q'] == '2024Q3']
credit_step3 = credit_step2[credit_step2['NomeColuna'] == 'Carteira de Crédito Classificada']

print(f"📊 Final filtered rows: {credit_step3.shape[0]:,}")

# Calculate the sum
sum_carteira_credito_2024Q3 = credit_step3['Saldo'].sum()
print(f"\n💳 CARTEIRA DE CRÉDITO 2024Q3: {sum_carteira_credito_2024Q3:,.2f}")
print(f"💳 In trillions: {sum_carteira_credito_2024Q3/1_000_000_000_000:.2f} trillion")
print(f"💳 In billions: {sum_carteira_credito_2024Q3/1_000_000_000:.2f} billion")

# Verify this matches user's calculation: 6,709,275,484,424.04
user_expected = 6709275484424.04
difference = abs(sum_carteira_credito_2024Q3 - user_expected)
print(f"\n✅ User expected: {user_expected:,.2f}")
print(f"🔍 Difference: {difference:,.2f} ({'✅ MATCH' if difference < 1 else '❌ MISMATCH'})")

🧮 Replicating user's Carteira de Crédito calculation:
Filter: NomeRelatorio == 'Resumo' AND AnoMes_Q == '2024Q3' AND NomeColuna == 'Carteira de Crédito Classificada'
📊 Final filtered rows: 1,395

💳 CARTEIRA DE CRÉDITO 2024Q3: 6,709,275,484,424.04
💳 In trillions: 6.71 trillion
💳 In billions: 6709.28 billion

✅ User expected: 6,709,275,484,424.04
🔍 Difference: 0.00 (✅ MATCH)


## 🗄️ Step 6: Connect to Database & Compare

In [8]:
# Connect to Supabase database
def get_database_connection():
    try:
        conn = psycopg2.connect(
            host="aws-1-sa-east-1.pooler.supabase.com",
            database="postgres",
            user="postgres.uwoxkycxkidipgbptsgx",
            password="En9QmRQaw14nhwxL",
            port="6543"
        )
        return conn
    except Exception as e:
        print(f"❌ Database connection failed: {str(e)}")
        return None

# Connect to database
conn = get_database_connection()
if conn:
    print("✅ Database connection successful")
    cursor = conn.cursor()

    # Check database structure
    cursor.execute("SELECT COUNT(*) FROM financial_data")
    total_records = cursor.fetchone()[0]
    print(f"📊 Total records in financial_data table: {total_records:,}")

    cursor.execute("SELECT COUNT(*) FROM institutions WHERE status = 'active'")
    active_institutions = cursor.fetchone()[0]
    print(f"🏦 Active institutions: {active_institutions:,}")

    cursor.execute("SELECT COUNT(*) FROM metrics")
    total_metrics = cursor.fetchone()[0]
    print(f"📈 Total metrics: {total_metrics:,}")
else:
    print("❌ Could not connect to database")

✅ Database connection successful
📊 Total records in financial_data table: 372,402
🏦 Active institutions: 50
📈 Total metrics: 158


## 🔍 Step 7: Investigate Database Calculations

In [9]:
if conn:
    print("🔍 Investigating current database calculations:")

    # Get latest quarter data as API does
    cursor.execute("""
        SELECT year, quarter, quarter_text
        FROM time_periods tp
        WHERE EXISTS (
            SELECT 1 FROM financial_data fd WHERE fd.time_period_id = tp.id
        )
        ORDER BY year DESC, quarter DESC
        LIMIT 1
    """)

    latest_q = cursor.fetchone()
    print(f"📅 Latest quarter in database: {latest_q[0]}Q{latest_q[1]} ({latest_q[2]})")

    # Replicate current API query for Ativo Total
    cursor.execute("""
        SELECT
            SUM(CASE WHEN m.nome_coluna = 'Ativo Total' THEN fd.valor END) as total_assets,
            COUNT(CASE WHEN m.nome_coluna = 'Ativo Total' THEN 1 END) as asset_records,
            COUNT(DISTINCT CASE WHEN m.nome_coluna = 'Ativo Total' THEN i.id END) as institutions_with_assets
        FROM financial_data fd
        JOIN institutions i ON fd.institution_id = i.id
        JOIN metrics m ON fd.metric_id = m.id
        JOIN time_periods tp ON fd.time_period_id = tp.id
        WHERE tp.year = %s AND tp.quarter = %s
          AND m.nome_coluna = 'Ativo Total'
          AND i.status = 'active'
          AND fd.valor IS NOT NULL
    """, (latest_q[0], latest_q[1]))

    db_assets = cursor.fetchone()
    print(f"\n💰 DATABASE ATIVO TOTAL {latest_q[0]}Q{latest_q[1]}:")
    print(f"   Value: {db_assets[0]:,.2f}")
    print(f"   Records: {db_assets[1]:,}")
    print(f"   Institutions: {db_assets[2]:,}")
    print(f"   In billions: {db_assets[0]/1_000_000_000:.2f} billion")

    # Compare with CSV calculation
    csv_value = sumativo_total_2024Q3
    db_value = db_assets[0] or 0
    print(f"\n🔍 COMPARISON:")
    print(f"   CSV calculation: {csv_value:,.2f}")
    print(f"   Database result: {db_value:,.2f}")
    print(f"   Difference: {abs(csv_value - db_value):,.2f}")
    print(f"   Ratio: {db_value/csv_value*100:.2f}% of expected" if csv_value > 0 else "N/A")

🔍 Investigating current database calculations:


OperationalError: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.


## 🔬 Step 8: Detailed ETL Investigation

In [10]:
if conn:
    print("🔬 Investigating ETL process and data mapping:")

    # Check metrics table for 'Ativo Total'
    cursor.execute("""
        SELECT id, codigo, nome_coluna, tipo_dado, categoria, description
        FROM metrics
        WHERE nome_coluna = 'Ativo Total'
        ORDER BY id
    """)

    ativo_metrics = cursor.fetchall()
    print(f"\n📊 Metrics table entries for 'Ativo Total': {len(ativo_metrics)}")
    for metric in ativo_metrics:
        print(f"   ID: {metric[0]}, Code: {metric[1]}, Name: {metric[2]}")

    # Check institutions mapping
    cursor.execute("""
        SELECT COUNT(*) as total,
               COUNT(CASE WHEN status = 'active' THEN 1 END) as active,
               COUNT(CASE WHEN status = 'inactive' THEN 1 END) as inactive
        FROM institutions
    """)

    inst_counts = cursor.fetchone()
    print(f"\n🏦 Institutions status:")
    print(f"   Total: {inst_counts[0]:,}")
    print(f"   Active: {inst_counts[1]:,}")
    print(f"   Inactive: {inst_counts[2]:,}")

    # Sample some data from financial_data for latest quarter
    cursor.execute("""
        SELECT i.name, m.nome_coluna, fd.valor
        FROM financial_data fd
        JOIN institutions i ON fd.institution_id = i.id
        JOIN metrics m ON fd.metric_id = m.id
        JOIN time_periods tp ON fd.time_period_id = tp.id
        WHERE tp.year = %s AND tp.quarter = %s
          AND m.nome_coluna = 'Ativo Total'
          AND i.status = 'active'
          AND fd.valor IS NOT NULL
        ORDER BY fd.valor DESC
        LIMIT 10
    """, (latest_q[0], latest_q[1]))

    sample_data = cursor.fetchall()
    print(f"\n🔍 Top 10 institutions by Ativo Total in database:")
    for row in sample_data:
        print(f"   {row[0][:30]:<30}: {row[2]:>15,.2f}")

🔬 Investigating ETL process and data mapping:


InterfaceError: cursor already closed

## 📋 Step 9: Check CSV vs Database Institution Mapping

In [ ]:
# Compare institution names and values between CSV and database
print("🔍 Comparing institutions between CSV and Database:")

# Get CSV institutions for 2024Q3 Ativo Total
csv_institutions = step3.groupby('InstituicaoFinanceira')['Saldo'].sum().sort_values(ascending=False)
print(f"\n📊 CSV: {len(csv_institutions)} institutions with Ativo Total data")
print("Top 10 institutions in CSV:")
for i, (inst, value) in enumerate(csv_institutions.head(10).items()):
    print(f"   {i+1:2d}. {inst[:40]:<40}: {value:>15,.2f}")

if conn and sample_data:
    print(f"\n📊 Database: {len(sample_data)} institutions shown (limited to 10)")
    print("Top 10 institutions in Database:")
    for i, row in enumerate(sample_data):
        print(f"   {i+1:2d}. {row[0][:40]:<40}: {row[2]:>15,.2f}")

    # Check if names match
    csv_names = set(csv_institutions.head(10).index)
    db_names = set([row[0] for row in sample_data])

    print(f"\n🔍 Name matching analysis:")
    print(f"   Common names: {len(csv_names.intersection(db_names))}")
    print(f"   CSV only: {len(csv_names - db_names)}")
    print(f"   Database only: {len(db_names - csv_names)}")

    if csv_names - db_names:
        print(f"\n   Names in CSV but not in DB top 10:")
        for name in list(csv_names - db_names)[:5]:
            print(f"     - {name}")

    if db_names - csv_names:
        print(f"\n   Names in DB but not in CSV top 10:")
        for name in list(db_names - csv_names)[:5]:
            print(f"     - {name}")

## 🚨 Step 10: Problem Identification & Root Cause Analysis

In [11]:
print("🚨 ROOT CAUSE ANALYSIS SUMMARY:")
print("=" * 60)

print(f"\n📊 EXPECTED VALUES (from CSV direct calculation):")
print(f"   Ativo Total 2024Q3: {sumativo_total_2024Q3:,.2f} ({sumativo_total_2024Q3/1_000_000_000:.1f} billion)")
print(f"   Credit Portfolio 2024Q3: {sum_carteira_credito_2024Q3:,.2f} ({sum_carteira_credito_2024Q3/1_000_000_000:.1f} billion)")

if conn and db_assets:
    print(f"\n🗄️ ACTUAL VALUES (from Database API):")
    print(f"   Ativo Total 2024Q3: {db_assets[0]:,.2f} ({db_assets[0]/1_000_000_000:.1f} billion)")

    ratio = (db_assets[0] / sumativo_total_2024Q3) * 100 if sumativo_total_2024Q3 > 0 else 0
    print(f"\n❌ DISCREPANCY:")
    print(f"   Database shows only {ratio:.1f}% of expected value")
    print(f"   Missing: {sumativo_total_2024Q3 - db_assets[0]:,.2f} ({(sumativo_total_2024Q3 - db_assets[0])/1_000_000_000:.1f} billion)")

print(f"\n🔍 POTENTIAL ISSUES TO INVESTIGATE:")
print(f"   1. ETL filtering: Are all institutions being loaded?")
print(f"   2. Data mapping: Are institution names mapped correctly?")
print(f"   3. Metric mapping: Is 'Ativo Total' metric ID correct?")
print(f"   4. Status filtering: Are institutions incorrectly marked as inactive?")
print(f"   5. Quarter mapping: Is 2024Q3 correctly loaded as time_period?")
print(f"   6. Value conversion: Are Saldo values being converted correctly?")
print(f"   7. Duplicate handling: Are records being deduplicated incorrectly?")

print(f"\n✅ NEXT STEPS:")
print(f"   1. Audit ETL process step by step")
print(f"   2. Compare record counts at each ETL stage")
print(f"   3. Verify institution and metric mappings")
print(f"   4. Test with smaller dataset to isolate issue")
print(f"   5. Fix database/API calculations")
print(f"   6. Revalidate frontend data display")

🚨 ROOT CAUSE ANALYSIS SUMMARY:

📊 EXPECTED VALUES (from CSV direct calculation):
   Ativo Total 2024Q3: 16,906,576,545,732.06 (16906.6 billion)
   Credit Portfolio 2024Q3: 6,709,275,484,424.04 (6709.3 billion)


NameError: name 'db_assets' is not defined

## 📋 Step 11: Data Quality Metrics

In [12]:
print("📊 DATA QUALITY METRICS:")
print("=" * 40)

# CSV data quality
print(f"\n📁 CSV FILE ANALYSIS:")
print(f"   Total records: {financial_metrics.shape[0]:,}")
print(f"   Unique institutions: {financial_metrics['InstituicaoFinanceira'].nunique():,}")
print(f"   Unique quarters: {financial_metrics['AnoMes_Q'].nunique():,}")
print(f"   Unique metrics: {financial_metrics['NomeColuna'].nunique():,}")

# Focus on 2024Q3 data
q3_data = financial_metrics[financial_metrics['AnoMes_Q'] == '2024Q3']
print(f"\n📅 2024Q3 SPECIFIC DATA:")
print(f"   Total records: {q3_data.shape[0]:,}")
print(f"   Institutions: {q3_data['InstituicaoFinanceira'].nunique():,}")
print(f"   Metrics: {q3_data['NomeColuna'].nunique():,}")
print(f"   Reports: {q3_data['NomeRelatorio'].value_counts().to_dict()}")

# Check for missing values
print(f"\n🔍 MISSING VALUES ANALYSIS:")
missing_analysis = q3_data.isnull().sum()
for col, missing in missing_analysis.items():
    if missing > 0:
        percentage = (missing / len(q3_data)) * 100
        print(f"   {col}: {missing:,} ({percentage:.1f}%)")

# Value ranges for key metrics
ativo_data = q3_data[(q3_data['NomeRelatorio'] == 'Resumo') & (q3_data['NomeColuna'] == 'Ativo Total')]
print(f"\n💰 ATIVO TOTAL VALUE ANALYSIS (2024Q3):")
print(f"   Records: {len(ativo_data):,}")
print(f"   Min value: {ativo_data['Saldo'].min():,.2f}")
print(f"   Max value: {ativo_data['Saldo'].max():,.2f}")
print(f"   Mean value: {ativo_data['Saldo'].mean():,.2f}")
print(f"   Median value: {ativo_data['Saldo'].median():,.2f}")
print(f"   Total sum: {ativo_data['Saldo'].sum():,.2f}")

if conn:
    conn.close()
    print("\n✅ Database connection closed")

📊 DATA QUALITY METRICS:

📁 CSV FILE ANALYSIS:
   Total records: 2,682,551


KeyError: 'InstituicaoFinanceira'

---
## 🎯 CONCLUSION

This notebook has identified a **massive discrepancy** between the expected values from direct CSV calculations and the values returned by our database API.

**Key Findings:**
- CSV shows **~16.9 trillion** in total assets for 2024Q3
- Database API returns only **~148 billion** (less than 1% of expected)
- The ETL process or database queries have fundamental issues

**Next Actions Required:**
1. Review and fix ETL data loading process
2. Verify database schema and data mappings
3. Test API calculations against known values
4. Update frontend once calculations are corrected

**Impact:**
- All current API endpoints return incorrect data
- Frontend displays are showing wrong values
- User trust and data accuracy is compromised

---